# 回归

- 包含线性回归、多项式回归，岭回归，lasso等，由sklearn来实现

---

## 线性回归

### 1.1 数据导入

- 数据集介绍及划分<br>波士顿房价数据集是机器学习中非常知名的数据集，它被用于多篇回归算法研究的学术论文中。该数据集共计 `506` 条，其中包含有 `13` 个与房价相关的特征以及 `1` 个目标值（房价）。

首先，我们使用 Pandas 加载并预览数据集。数据集名称为 `course-5-boston.csv`。

In [3]:
!wget http://labfile.oss.aliyuncs.com/courses/1081/course-5-boston.csv

--2018-10-21 06:40:10--  http://labfile.oss.aliyuncs.com/courses/1081/course-5-boston.csv
Resolving labfile.oss.aliyuncs.com (labfile.oss.aliyuncs.com)... 118.178.161.16
Connecting to labfile.oss.aliyuncs.com (labfile.oss.aliyuncs.com)|118.178.161.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35203 (34K) [text/csv]
Saving to: ‘course-5-boston.csv’

course-5-boston.csv 100%[===================>]  34.38K  --.-KB/s    in 0.06s   

2018-10-21 06:40:10 (571 KB/s) - ‘course-5-boston.csv’ saved [35203/35203]



In [5]:
import pandas as pd

df = pd.read_csv('course-5-boston.csv')
df.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


该数据集统计了波士顿地区各城镇的住房价格中位数，以及与之相关的特征。每列数据的列名解释如下：

- `CRIM`: 城镇犯罪率。
- `ZN`: 占地面积超过 2.5 万平方英尺的住宅用地比例。
- `INDUS`: 城镇非零售业务地区的比例。
- `CHAS`: 查尔斯河是否经过 (`=1` 经过，`=0` 不经过)。
- `NOX`: 一氧化氮浓度（每 `1000` 万份）。
- `RM`: 住宅平均房间数。
- `AGE`: 所有者年龄。
- `DIS`: 与就业中心的距离。
- `RAD`: 公路可达性指数。
- `TAX`: 物业税率。
- `PTRATIO`: 城镇师生比例。
- `BLACK`: 城镇的黑人指数。
- `LSTAT`: 人口中地位较低人群的百分数。
- `MEDV`: 城镇住房价格中位数。

本次实验中，我们不会使用到全部的数据特征。这里，仅选取 `CRIM`, `RM`, `LSTAT` 三个特征用于线性回归模型训练。我们将这三个特征的数据单独拿出来，并且使用 `describe()` 方法查看其描述信息。 `describe()` 统计了每列数据的个数、最大值、最小值、平均数等信息。

In [11]:
data = df[['crim','rm','lstat']]
target = df['medv']

### 1.2 数据集划分

In [35]:
# 划分数据集，分为train,test data
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(data,target,test_size = 0.3, random_state=1)

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(354, 3) (152, 3) (354,) (152,)


### 1.3 训练模型

```python
sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
```

- `fit_intercept`: 默认为 True，计算截距项。
- `normalize`: 默认为 False，不针对数据进行标准化处理。
- `copy_X`: 默认为 True，即使用数据的副本进行操作，防止影响原数据。
- `n_jobs`: 计算时的作业数量。默认为 1，若为 -1 则使用全部 CPU 参与运算。

In [36]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [37]:
# 得到模型拟合参数
model.intercept_,model.coef_

(4.711271536469873, array([-0.07482133,  4.17449329, -0.6325041 ]))

### 1.4 测试数据并模型评估

In [38]:
preds = model.predict(x_test)

In [43]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

mae = mean_absolute_error(y_pred=preds,y_true=y_test)
mse = mean_squared_error(y_pred=preds,y_true=y_test)
print(mae,mse)

4.13360936127 27.88953417


---

## 多项式回归

思路：是将数据转化为产生多项式特征矩阵，再用LR去做回归

### 2.1 导入数据并预览